In [23]:
import numpy as np


In [24]:
r = 0.01 #m  1 cm

def volume(r):
    V = (4/3) * np.pi * r**3
    return V

alpha = np.pi/8 # 22.5 degrees in radians
z = 0.05 # m
x = z*np.sin(alpha)
print(x)

0.01913417161825449


# resolvable object size

In [ ]:
c = 343  # m/s
f = 40e3  # Hz 
distance = 0.05  # m
wavelength = c / f
aperture_diameter = 0.01  # m laterael resloution does not matter? why? works but bad!! look for distanc reoluiotn!
def resolvable_size(wavelength, aperture_diameter, distance):
    theta_min = (1.22 * wavelength / aperture_diameter)
    d = distance * (theta_min)
    return d

d = resolvable_size(wavelength, aperture_diameter, distance)

print(f"Resolvabel object size at {f/1e3} kHz: {d*1e2} cm")

def travel_time(c, distance):
    t = 2*distance / c
    return t
travel_t = travel_time(c, distance)  #10 cm depth
print(f"Travel time to {distance*1e2} cm depth and back: {travel_t*1e3} ms")

Resolvabel object size at 40.0 kHz: 5.23075 cm
Travel time to 5.0 cm depth and back: 0.2915451895043732 ms


# processing speed 
Real-time MCUs often have DMA to move ADC samples to buffer without CPU intervention.  
MCU and ADC have different clocks!     
Example (ATmega328P, Arduino Uno):  
CPU: 16 MHz  
ADC clock = CPU clock / prescaler  
Prescaler options: 2, 4, 8, 16, 32, 64, 128 

In [33]:
buffer_size = 256  #dont store whole echo in on buffer loading for real time feed back!  typical 64–1024 samples
adc_resolution = 16  #bits #for doppler maybe required! which frequency component is lost be not discretizing more coarlelsy? not answered typical 8–12 bits
cpu_clock_speed = 16e6  #Hz  #typical 1 MHz – 16 MHz
sampling_frequency = cpu_clock_speed/128 #kHz  #for 40kHz signal! Nyquist! ADC clock  typcal 1 kHz – 1 MHz
print(f"Sampling frequency: {sampling_frequency/1e3} kHz")

def adc_speed(buffer_size, adc_resolution, sampling_frequency):
    m = (buffer_size/adc_resolution) #amount of samples in buffer
    t = 1/sampling_frequency  #time per sample
    adc_time = m * t
    return adc_time



adc_time = adc_speed(buffer_size, adc_resolution, sampling_frequency)
print(f"Processing time for buffer size {buffer_size} at {sampling_frequency/1e3} kHz sampling frequency: {adc_time*1e3} ms")
    

def run_time(buffer_size, kernel_size, cpu_clock_speed):  #take into account how many cycles per operation are done!
    filter = buffer_size * kernel_size *(1 + 2 )  #O(n*m) (add + multiply)
    peak_detection_time = buffer_size   #O/n)  #comparisons take 1 cycle
    time_of_flight= 20 #estimated cycles
    delta_x =  20 #estimated cycles
    pid = 3 + 1+ 20  #controller    3 adds + 1 multiply (prestoreing Ks) + 1 division
    total_cycles = filter + peak_detection_time + time_of_flight + delta_x + pid
    total_cpu_time = total_cycles/cpu_clock_speed
    return total_cpu_time

total_cpu_time = run_time(buffer_size, kernel_size=5, cpu_clock_speed=cpu_clock_speed)
print(f"Estimated total processing time for buffer size {buffer_size} at {cpu_clock_speed/1e6} MHz CPU clock: {total_cpu_time*1e3} ms")


Sampling frequency: 125.0 kHz
Processing time for buffer size 256 at 125.0 kHz sampling frequency: 0.128 ms
Estimated total processing time for buffer size 256 at 16.0 MHz CPU clock: 0.25999999999999995 ms


# total control cycle speed:
neglet dac because its mycroseconds


In [34]:
def control_cycle_speed(travel_t, adc_time, total_cpu_time):
    sound_production = 0.25e-3  #5 cycles until resonance
    control_time = sound_production + travel_t + adc_time + total_cpu_time
    return control_time

control_time = control_cycle_speed( travel_t=travel_t, adc_time=adc_time, total_cpu_time=total_cpu_time)
print(f"Estimated total control cycle time: {control_time*1e3} ms")

Estimated total control cycle time: 0.929545189504373 ms
